# Demo

In [1]:
import sounddevice as sd
import numpy as np
import librosa
from scipy.io import wavfile as wav #import write

import cv2 as cv
import matplotlib.pyplot as plt

In [ ]:
import keras.models
from keras.preprocessing import image as kimage
from keras.applications  import vgg16

In [ ]:
# Caricamento modello per il rilevamento di volti frontali
face_detector = cv.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
# caricamento modelli per audio e per video
best_model_1 = tf.keras.models.load_model(dir_drive_model + "best_model")
net = keras.models.load_model('mymodel_64ba.h5')

In [ ]:
# classi
names = ['Marco' ,'Roberto'] #['Marco' ,'Roberto', 'Unknown']

In [ ]:
def process_frame(img):
    # Gray scale
    f_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    # Rilevamento volti
    faces = face_detector.detectMultiScale(f_gray) 
    # Sovrapposizione bounding box con nome all'immagine
    for (x,y,w,h) in faces:
        # box
        cv.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2) 
        
        face = image[y:y+h, x:x+w, :]
        img_pixels = cv.resize(face, (224, 224)) 
        img_pixels = kimage.img_to_array(img_pixels)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels = vgg16.preprocess_input(img_pixels)
        y_pred = net.predict(img_pixels)
        y_pred_name = np.argmax(y_pred)

        cv.putText(image, 
                   (names[y_pred_name]), #testo
                   (x+5,y-5),   #posizione
                   cv.FONT_HERSHEY_SIMPLEX, #font
                   1.5,         #fontScale
                   (255,0,255), #colore
                   2)           #spessore
    return image

In [ ]:
fs = 44000
sampling_rate=44000 
n_mfcc = 15 

eps     = 0.001
mean_e  = np.load('mean.npy')
std_e   = np.load('std.npy')

def extr_feat(sound_clip):
    # Mel Frequency Cepstral Coefficents
    mfcc = librosa.feature.mfcc(y=sound_clip, sr=sampling_rate,n_mfcc=n_mfcc)
            
    # MFCC deltas
    mfcc_delta = librosa.feature.delta(mfcc)
  
    # MFCC double deltas
    mfcc_delta2 = librosa.feature.delta(mfcc, order=2)
  
    mel_spectogram = librosa.feature.melspectrogram(y=sound_clip, sr=sampling_rate)
    # Root Mean Square Energy
    rmse = librosa.feature.rmse(S=mel_spectogram)
  
    mfcc = np.asarray(mfcc)
    mfcc_delta = np.asarray(mfcc_delta)
    mfcc_delta2 = np.asarray(mfcc_delta2)
    rmse = np.asarray(rmse)

    feature = np.concatenate((mfcc, mfcc_delta, mfcc_delta2, rmse), axis=0).T
    feature = np.asarray(feature)
    
    return feature

In [2]:
cap     = cv.VideoCapture(0)
seconds = 3 
fs      = 44100
sent    ='Premere "r" per riconoscere chi sta parlando'
cv.startWindowThread()

while(True):
    r, frame = cap.read()
    frame = process_frame(frame)
    cv.putText(frame, sent, (15, 37), cv.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1)  
    cv.rectangle(frame, (1, 1), (850, 45), (255,255,255), 2)
    cv.imshow('Video', frame)
    
    if cv.waitKey(20) & 0xFF == ord('r'):
        prova = sd.rec(int(seconds * fs), samplerate=fs, channels=1, blocking=True)
        wav.write('test.wav', rate=fs, data=(prova))
        _, rec = wav.read('test.wav')
        
        feature = extr_feat(rec)
        feature = np.array([row for row in (feature - mean_e[0:258] + eps)/(std_e[0:258] + eps)])
        feature = feature.reshape(1, feature.shape[0], feature.shape[1])
        
        prob_audio = best_model_1.predict(feature)
        pred_audio = np.argmax(prob_audio)
        
        if   pred_audio == 0:
            res = "Roberto"

        elif pred_audio == 1:
            res = "Marco"

        elif pred_audio == 2:
            res = "Sconosciuto"

        else:
            res ="Retry"

        sent = 'Ha parlato '+str(res)+' '+str(prob_audio[0][pred_audio])


    if cv.waitKey(20) & 0xFF == ord("q"):
        break
        
cap.release()
cv.destroyAllWindows()